In [130]:
import pandas as pd
import numpy as np
%run similarities.ipynb
from datetime import timedelta

import os
import sys

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [2]:
convert={'FIPS': lambda x: str(x)}
main_df = pd.read_csv("../processed_data/combined.csv", converters = convert)

In [3]:
main_df.FIPS = [str(item).zfill(5) for item in main_df.FIPS]
main_df['date'] = main_df['date'].astype(str) + '-20'
main_df['date'] = pd.to_datetime(main_df.date)

In [109]:
window_length = 2
window_start = pd.to_datetime("05-25-2020")
window_end = window_start + timedelta(days = window_length)

window_df = main_df[(main_df['date'] >= window_start ) & (main_df['date'] <= window_end)]

In [110]:
## temp to have something to subset by
window_df["temp"] = (np.random.randint(0,10, size=len(window_df)) > 8)*1

/Users/Sapthau/opt/anaconda3/envs/cs109a/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [111]:
window_df.reset_index(inplace = True, drop = True)
yes_df = window_df[window_df["temp"] == 1].copy()
no_df = window_df[window_df["temp"] == 0].copy()

yes_df.drop(["temp"], axis = 1, inplace = True)
yes_df.reset_index(inplace = True, drop = True)
no_df.drop(["temp"], axis = 1, inplace = True)
no_df.reset_index(inplace = True, drop = True)


comparables = ['population', 'female_percentage','life_expectancy',
       'mortality_risk','pct_none', 'pct_hs',
       'pct_bachelors', 'median_household_income', 'pct_black', 'pct_asian',
       'pct_hispanic', 'pct_non_hispanic_white',
       'pct_not_proficient_in_english', 'pct_rural', 'pct_impoverished',
       'pop_density', 'r_voteshare']

protests_df = yes_df[comparables].copy()
compare_df = no_df[comparables].copy()



In [112]:
def progressbar(n_step, n_total):
    """Prints self-updating progress bar to stdout to track for-loop progress
    
    There are entire 3rd-party libraries dedicated to custom progress-bars.
    A simple function like this is often more than enough to get the job done.
    
    :param n_total: total number of expected for-loop iterations
    :type n_total: int
    :param n_step: current iteration number, starting at 0
    :type n_step: int

    .. example::
    
        for i in range(n_iterations):
            progressbar(i, n_iterations)
            
    .. source:
    
        This function is a simplified version of code found here:
        https://stackoverflow.com/questions/3160699/python-progress-bar/15860757#15860757
    """
    n_step = n_step + 1
    barlen = 50
    progress = n_step / n_total
    block = int(round(barlen * progress))
    status = ""
    if n_step == n_total:
        status = "Done...\r\n\n"
    text = "\r [{0}] {1}/{2} {3}".format(
        "=" * block + "-" * (barlen - block),
        n_step,
        n_total,
        status,
    )
    sys.stdout.write(text)
    sys.stdout.flush()

In [137]:
nearest_idx = {}

for idx, row in yes_df.iterrows():
    closest = find_approx_closest_euc(compare_df[no_df.date == yes_df.date[idx]].copy(), row,["pct_hs"],[0.1], 10)
    nearest_idx[(yes_df.FIPS[idx], yes_df.date[idx])] = no_df.FIPS[closest].values
    progressbar(idx, yes_df.shape[0])
    
    

 [==================================================] 944/944 Done...

